## 📊 Optimización de Portafolio (Modelo de Markowitz)

Se desea encontrar la asignación óptima de inversión en 20 activos financieros (`A1, A2, A3, A4, A5, ..., A20`), de modo que se **minimice el riesgo total del portafolio** utilizando la matriz de covarianza de rendimientos históricos.

---

### 🔸 Variables de decisión

- $w_i$: fracción del capital total invertida en el activo $i$

---

### 🔸 Función Objetivo

Minimizar la varianza del portafolio:

$$
\min \quad w^T \Sigma w = \sum_i \sum_j w_i \cdot \Sigma_{ij} \cdot w_j
$$

---

### 🔸 Restricciones

1. Presupuesto total (todo el capital debe invertirse):

$$
\sum_i w_i = 1
$$

2. No se permite venta corta:

$$
w_i \geq 0 \quad \forall i
$$

---

### 💻 Implementación en Pyomo



In [1]:
from pyomo.environ import *
import pandas as pd

# Cargar la matriz de covarianza
df_cov = pd.read_csv(r"D:\AminadabCA\MIAAD\Analítica_Prescriptiva\Ejercicios_casos\Caso_5Matriz_Covarianza_Portafolio.csv", index_col=0)
assets = df_cov.columns.tolist()
n = len(assets)

# Crear el modelo
model = ConcreteModel()
model.Assets = Set(initialize=assets)
model.w = Var(model.Assets, bounds=(0, 1))  # pesos entre 0 y 1

# Restricción: suma de los pesos = 1
model.budget = Constraint(expr=sum(model.w[i] for i in model.Assets) == 1)

# Parámetro de covarianza
cov_dict = {
    (assets[i], assets[j]): df_cov.iloc[i, j]
    for i in range(n) for j in range(n)
}
model.Sigma = Param(model.Assets, model.Assets, initialize=cov_dict)

# Función objetivo: minimizar la varianza del portafolio
def variance_rule(m):
    return sum(m.w[i] * m.Sigma[i, j] * m.w[j] for i in m.Assets for j in m.Assets)
model.obj = Objective(rule=variance_rule, sense=minimize)

# Resolver
solver = SolverFactory('ipopt')
results = solver.solve(model, tee=True)

# Mostrar resultados
for i in model.Assets:
    print(f"{i}: {value(model.w[i]):.4f}")
print("Varianza mínima del portafolio:", value(model.obj))


Ipopt 3.12.13: 

******************************************************************************
This program contains Ipopt, a library for large-scale nonlinear optimization.
 Ipopt is released as open source code under the Eclipse Public License (EPL).
         For more information visit http://projects.coin-or.org/Ipopt
******************************************************************************

This is Ipopt version 3.12.13, running with linear solver mumps.
NOTE: Other linear solvers might be more efficient (see Ipopt documentation).

Number of nonzeros in equality constraint Jacobian...:       20
Number of nonzeros in inequality constraint Jacobian.:        0
Number of nonzeros in Lagrangian Hessian.............:      210

Total number of variables............................:       20
                     variables with only lower bounds:        0
                variables with lower and upper bounds:       20
                     variables with only upper bounds:        0
Tot